# Hidden Markov state models (HMMs)

This notebook is a shorted version of the notebook about HMM in the PyEMMA tutorials <a href="https://github.com/markovmodel/pyemma_tutorials">available here</a>. 
<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Licence" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" title='This work is licensed under a Creative Commons Attribution 4.0 International License.' align="right"/></a>

In this notebook, we will learn about hidden Markov state models and how to use them to deal with poor discretization.
We further explain how to obtain a coarse-grained model based on an initial MSM analysis.

**Remember**:
- to run the currently highlighted cell, hold <kbd>&#x21E7; Shift</kbd> and press <kbd>&#x23ce; Enter</kbd>;
- to get help for a specific function, place the cursor within the function's brackets, hold <kbd>&#x21E7; Shift</kbd>, and press <kbd>&#x21E5; Tab</kbd>;
- you can find the full documentation at [PyEMMA.org](http://www.pyemma.org).

---

⚠️ Please note that the PyEMMA implementation internally initiates the estimation with a regular MSM and PCCA++.

There are two major use-cases for HMMs in PyEMMA:
- HMMs are more robust to poor space discretization and can be used to overcome difficult clustering situations
- HMMs offer a coarse graining into metastable (hidden) states

In this notebook, we will demonstrate how to estimate HMMs and how they behave in comparison to MSMs.

⚠️ We have assigned the integer numbers $1 \dots $ `nstates` to HMM metastable states.
As PyEMMA is written in Python, it internally indexes states starting from $0$.
In consequence, numbers in the code cells differ by $-1$ from the plot labels and markdown text. 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import mdshare
import pyemma

## Case 1: preprocessed, two-dimensional data (toy model)

In this example, we are going to demonstrate the robustness of HMMs against poor discretization and show some of its properties.
We start by loading the two-dimensional data as well as the true discrete trajectory from an archive using numpy:

In [ ]:
file = mdshare.fetch('hmm-doublewell-2d-100k.npz', working_directory='data')
with np.load(file) as fh:
    data = fh['trajectory']
    good_dtraj = fh['discrete_trajectory']

We now estimate a reference (regular) MSM from the well-discretized data which is shown in the next panel (left).
We include an implied timescales plot.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
pyemma.plots.plot_state_map(*data.T, good_dtraj, ax=axes[0])
axes[0].scatter(*np.asarray([[0, -1], [0, 1]]).T, s=15, c='C1')

axes[0].set_xlabel('$x$')
axes[0].set_xlim(-4, 4)
axes[0].set_ylim(-4, 4)
axes[0].set_aspect('equal')
axes[0].set_ylabel('$y$')
axes[0].set_title('discretization')

lags = [i for i in range(1, 10)]
pyemma.plots.plot_implied_timescales(
    pyemma.msm.its(good_dtraj, lags=lags, errors='bayes'), ylog=False, ax=axes[1])
axes[1].set_title('MSM with good discretization')
fig.tight_layout()

We note that for this very good discretization, the implied timescales are converged  from lagtime $1$ step.
We continue to build an MSM object and perform the Chapman-Kolmogorov test:

In [ ]:
reference_msm = pyemma.msm.estimate_markov_model(good_dtraj, lag=1)
pyemma.plots.plot_cktest(reference_msm.cktest(2));

The Chapman-Kolmogorov test shows excellent agreement between higher lagtime estimation and model prediction.
We thus take this model as a reference.

Let's now deliberately choose a very bad discretization...

In [ ]:
poor_clustercenters = np.asarray([[-2.5, -1.4], 
                                  [0.3, 1.2], 
                                  [2.7, -0.6]])
poor_dtraj = pyemma.coordinates.assign_to_centers(data, centers=poor_clustercenters)[0]

... and repeat the ITS estimation:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
pyemma.plots.plot_state_map(*data.T, poor_dtraj, ax=axes[0])
axes[0].scatter(*poor_clustercenters.T, s=15, c='C1')

axes[0].set_xlabel('$x$')
axes[0].set_xlim(-4, 4)
axes[0].set_ylim(-4, 4)
axes[0].set_aspect('equal')
axes[0].set_ylabel('$y$')
axes[0].set_title('discretization')

pyemma.plots.plot_implied_timescales(
    pyemma.msm.its(poor_dtraj, lags=lags, errors='bayes'), ylog=False, ax=axes[1])
axes[1].set_title('MSM with poor discretization')
fig.tight_layout()

Obviously, the discretization is very poor and does not mirror the basins of the double-well potential anymore.
All three discrete states include data points from the two metastable regions (left panel) and,
as the right panel shows, this discretization error cannot be fixed by using a large lagtime for a regular MSM estimation.
Thus, the MSM clearly is not able to resolve the slow process connecting the two basins.

⚠️ We do not see any ITS above the lag time horizon and, hence, cannot estimate any MSM with this discretization.

Let us now repeat both estimations, using well and poorly discretized data, with hidden Markov models instead of regular MSMs.
We begin with the implied timescale convergence using the `pyemma.msm.timescales_hmsm()` function and two hidden states:

In [ ]:
its_hmm_poor = pyemma.msm.timescales_hmsm(poor_dtraj, nstates=2, lags=lags, errors='bayes')
its_hmm_good = pyemma.msm.timescales_hmsm(good_dtraj, nstates=2, lags=lags, errors='bayes')

We go on visualizing the results as with regular MSM implied timescales and include, as a dotted line,
the result from our previously estimated reference MSM. 

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharey=True)
pyemma.plots.plot_implied_timescales(
    its_hmm_poor,
    ylog=False, ax=axes[0])
pyemma.plots.plot_implied_timescales(
    its_hmm_good,
    ylog=False, ax=axes[1])
axes[0].set_title('HMM with poor discretization')
axes[1].set_title('HMM with good discretization')

for n, ax in enumerate(axes.flat):
    ax.set_ylim(-0.5, 12.5)
    ax.hlines(reference_msm.timescales()[0], *ax.get_xlim(), linestyle=':', label='reference MSM' if n == 0 else None)
fig.legend(loc=9)
fig.tight_layout()

In contrast to a regular MSM, both discretizations give us converged implied timescales from the very start (lagtime $1$ step).
The only difference is that the poor discretization yields a larger error and we loose the process faster.
As the HMM computes the implied timescales of a process between two hidden states,
we do not assume Markovianity in the original state space.
Thus, the deliberate discretization error we made is compensated by the algorithm,
making it robust against poor clustering.

In order to validate this claim, we estimate HMMs using both discretizations at lagtime $1$ step and two hidden states...

In [ ]:
poor_hmm = pyemma.msm.estimate_hidden_markov_model(poor_dtraj, 2, lag=1)
good_hmm = pyemma.msm.estimate_hidden_markov_model(good_dtraj, 2, lag=1)

print('MSM (ref):  1. implied timescale = {:.2f} steps'.format(reference_msm.timescales()[0]))
print('HMM (poor): 1. implied timescale = {:.2f} steps'.format(poor_hmm.timescales()[0]))
print('HMM (good): 1. implied timescale = {:.2f} steps'.format(good_hmm.timescales()[0]))

... and obtain nearly identical estimates for the first implied timescale that agree with the reference MSM.

We observe that HMMs, unlike MSMs, seem to be somewhat resistant to discretization errors.

Regarding the CK test, we again see that the `poor_hmm`...

In [ ]:
pyemma.plots.plot_cktest(poor_hmm.cktest(2));

... and the `good_hmm`...

In [ ]:
pyemma.plots.plot_cktest(good_hmm.cktest(2));

... are in perfect agreement and the Chapman-Kolmogorov test has passed.

The HMM, even for the poor discretization, learns an assignment between the microstates to the hidden states that reproduces the true dynamics better than MSMs.
We can extract this information from the HMM object using its `hmm.hidden_state_probabilities` property.
It contains the probabilities for each microstate to be in a given hidden state over time,
for each trajectory (which is why we have to take the $0$-th element from this list). 

Let's now plot the hidden state probabilities in the original space:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
for i, ax in enumerate(axes.flat):
    pyemma.plots.plot_contour(
        *data.T,
        poor_hmm.hidden_state_probabilities[0][:, i], # index 0: only 1 trajectory
        ax=ax,
        cmap='afmhot_r', 
        mask=True,
        cbar_label='P(state {})'.format(i+1))
    ax.set_xlabel('$y')
axes[0].set_ylabel('$x$')
fig.tight_layout()

Even though we find some artifacts of the initial clustering,
the HMM has basically overcome the poor discretization and found hidden states that,
with high certainty, mirror the original double-well basins.

Let's finish this example by comparing the hidden state trajectories with the discrete trajectories that were used for estimating the reference MSM (the "true" discrete trajectories)...

In [ ]:
print('HMM (good): Hidden state trajectory consistency: '
      '{:.3f}'.format(sum(good_hmm.hidden_state_trajectories[0] == good_dtraj)/good_dtraj.shape[0]))
print('HMM (poor): Hidden state trajectory consistency: '
      '{:.3f}'.format(sum(poor_hmm.hidden_state_trajectories[0] == good_dtraj)/good_dtraj.shape[0]))

... and the stationary distributions of both HMMs to the reference MSM:

In [ ]:
print('MSM (ref):  stationary distribution = {}'.format(np.round(reference_msm.pi, 4)))
print('HMM (poor): stationary distribution = {}'.format(np.round(poor_hmm.pi, 4)))
print('HMM (good): stationary distribution = {}'.format(np.round(good_hmm.pi, 4)))

We find that even with a very poor discretization,
HMMs are capable of recovering the kinetics of the underlying process with very little error. 

## Case 2: low-dimensional molecular dynamics data (alanine dipeptide)

We are now illustrating a typical use case of hidden markov state models:
estimating an MSM that is used as a heuristics for the number of slow processes or hidden states,
and estimating an HMM (to overcome potential discretization issues and to resolve faster processes than an MSM).

We fetch the alanine dipeptide data set, load the backbone torsions into memory...

In [ ]:
pdb = mdshare.fetch('alanine-dipeptide-nowater.pdb', working_directory='data')
files = mdshare.fetch('alanine-dipeptide-*-250ns-nowater.xtc', working_directory='data')

feat = pyemma.coordinates.featurizer(pdb)
feat.add_backbone_torsions(periodic=False)
data = pyemma.coordinates.load(files, features=feat)
data_concatenated = np.concatenate(data)

cluster = pyemma.coordinates.cluster_kmeans(data, k=75, max_iter=50, stride=10)
dtrajs_concatenated = np.concatenate(cluster.dtrajs)

... discretize the full space using $k$-means clustering,
visualize the marginal and joint distributions of both components as well as the cluster centers,
and show the ITS convergence to help selecting a suitable lag time:

In [ ]:
its = pyemma.msm.its(
    cluster.dtrajs, lags=[1, 2, 5, 10, 20, 50], nits=4, errors='bayes')

fig, axes = plt.subplots(1, 3, figsize=(12, 3))
pyemma.plots.plot_feature_histograms(data_concatenated, feature_labels=['$\Phi$', '$\Psi$'], ax=axes[0])
pyemma.plots.plot_density(*data_concatenated.T, ax=axes[1], cbar=False, alpha=0.3)
axes[1].scatter(*cluster.clustercenters.T, s=15, c='C1')
axes[1].set_xlabel('$\Phi$')
axes[1].set_ylabel('$\Psi$')
pyemma.plots.plot_implied_timescales(its, ax=axes[2], units='ps')
fig.tight_layout()

Based on the implied timescale convergence plot, we choose a lagtime of $10$ ps.
We further find $3$ slow processes in the implied timescales plot,
meaning that we can expect $4$ metastable sets or hidden states.

First, we estimate a Bayesian MSM, and show the results of a CK test:

In [ ]:
bayesian_msm = pyemma.msm.bayesian_markov_model(cluster.dtrajs, lag=10, dt_traj='1 ps')

nstates = 4
pyemma.plots.plot_cktest(bayesian_msm.cktest(nstates), units='ps');

At this point, we have a (bayesian) MSM with $75$ discrete states and basic validation.
To obtain an HMM with only four states (the number for which we have validated our MSM),
we compute the implied timescales for HMMs with this number of hidden states. 

We repeat the ITS convergence analysis using (bayesian) HMMs and small lagtimes for a $4$-state HMM.
For demonstration purposes, we add the same analysis with a $6$-state HMM to visualize what happens if the number of states is not as clear as in this example.

We use `pyemma.msm.timescales_hmsm()` to compute implied timescales for HMMs -- note that we need to specify the number of hidden states with the `nstates` keyword argument.

Since this cell may take long to execute we simply show a plot of the results here -- Curious people can check the code here: [Supplementary Cells](#Supplementary-Cells)

![Plots](_misc/hmm_its_4and6_states.png)

The left panel shows that an HMM with four hidden states yields converged implied timescales from lagtime $1$.

The right panel, however, shows that an HMM with six hidden states and lagtime $1$ can resolve two additional processes.

Let us follow up on this and perform a CK test for a four state HMM at lagtime $1$...

In [ ]:
hmm_4 = pyemma.msm.bayesian_hidden_markov_model(cluster.dtrajs, nstates=4, lag=1, dt_traj='1 ps', nsamples=50)
pyemma.plots.plot_cktest(hmm_4.cktest(mlags=6), units='ps');

... and than the six state HMM at laggtime $1$ (we use `mlags=2` because we would loose the two fast processes at lagtimes $\geq3$):

Since this cell may take long to execute we simply show a plot of the results here -- Curious people can check the code here: [Supplementary Cells](#Supplementary-Cells)

![Plots](_misc/hmm_ck_6states.png)

In both cases, the CK test is passed.

If we now compare both metastable membership plots...

In [ ]:
hmm_6 = pyemma.msm.bayesian_hidden_markov_model(cluster.dtrajs, nstates=6, lag=1, dt_traj='1 ps', nsamples=50)
fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharex=True, sharey=True)
for hmm, ax in zip([hmm_4, hmm_6], axes.flat):
    _, _, misc = pyemma.plots.plot_state_map(
        *data_concatenated.T,
        hmm.metastable_assignments[dtrajs_concatenated], 
        ax=ax)
    ax.set_title('HMM with {} hidden states'.format(hmm.nstates))
    ax.set_xlabel('$\Phi$')
    misc['cbar'].set_ticklabels(range(1, hmm.nstates + 1))
axes[0].set_ylabel('$\Psi$')
fig.tight_layout()

... we see that the six state HMM is able to subdivide the two largest metastable states of the four state HMM and,
thus, gives us a more detailed view on the underlying system.
As one would have expected from the implied timescale plot,
the metastable dynamics is already well-described with $4$ hidden states.

Like with classical MSMs, we can further analyze properties of the HMM.
As an example, have a look at the transition paths and committor probabilities below. 

In [ ]:
A = [0]
B = [3]
flux = pyemma.msm.tpt(hmm_4, A, B)

highest_membership = hmm_4.metastable_distributions.argmax(1)
coarse_state_centers = cluster.clustercenters[hmm_4.observable_set[highest_membership]]

Please note one important difference to MSMs:
since HMMs operate directly on the hidden states,
we must not compute the flux between the `msm.metastable_sets` but between the lists of macrostate numbers,
e.g., instead of `A = msm.metastable_sets[0]` we set `A = [0]`. 

Let's now visualize the committor as before.
Does it look familiar?

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

pyemma.plots.plot_contour(
    *data_concatenated.T,
    flux.committor[hmm_4.metastable_assignments[dtrajs_concatenated]],
    cmap='brg',
    ax=ax,
    mask=True,
    cbar_label=r'committor 0 $\to$ 3',
    alpha=0.8,
    zorder=-1)

pyemma.plots.plot_flux(
    flux,
    coarse_state_centers,
    flux.stationary_distribution,
    ax=ax,
    show_committor=False,
    figpadding=0,
    show_frame=True,
    state_labels=['A','' ,'', 'B'], 
    arrow_label_format='%2.e / ps');

ax.set_xlabel('$\Phi$')
ax.set_ylabel('$\Psi$')
ax.set_xlim(data_concatenated[:, 0].min(), data_concatenated[:, 0].max())
ax.set_ylim(data_concatenated[:, 1].min(), data_concatenated[:, 1].max())
fig.tight_layout()

As we see, in addition to the properties described above, HMMs provide the same analysis tools as MSMs.

Let us now repeat this approach again for another featurization:
we already know that it is possible to resolve six metastable states (five slow processes) using an HMM estimated on a discretization of the backbone torsions.
Can you achieve the same level of resolution using heavy atom distances and a suitable TICA projection?

#### Exercise 1

Obtain the heavy atom distances, use TICA for dimension reduction, and discretize using a method of your choice.

In [ ]:
feat = #FIXME
feat. #FIXME
data = #FIXME

tica = #FIXME
tica_concatenated = np.concatenate(tica.get_output())

cluster = #FIXME
dtrajs_concatenated = #FIXME

its = pyemma.msm.its(
    cluster.dtrajs, lags=[1, 2, 5, 10, 20, 50], nits=4, errors='bayes')

fig, axes = plt.subplots(1, 3, figsize=(12, 3))
pyemma.plots.plot_feature_histograms(tica_concatenated, ax=axes[0])
pyemma.plots.plot_density(*tica_concatenated[:, :2].T, ax=axes[1], cbar=False, alpha=0.1)
axes[1].scatter(*cluster.clustercenters[:, :2].T, s=15, c='C1')
axes[1].set_xlabel('IC 1')
axes[1].set_ylabel('IC 2')
pyemma.plots.plot_implied_timescales(its, ax=axes[2], units='ps')
fig.tight_layout()

###### Solution

In [ ]:
feat = pyemma.coordinates.featurizer(pdb)
pairs = feat.pairs(feat.select_Heavy())
feat.add_distances(pairs, periodic=False)
data = pyemma.coordinates.load(files, features=feat)

tica = pyemma.coordinates.tica(data, lag=3)
tica_concatenated = np.concatenate(tica.get_output())

cluster = pyemma.coordinates.cluster_kmeans(tica, k=75, max_iter=50, stride=10)
dtrajs_concatenated = np.concatenate(cluster.dtrajs)

its = pyemma.msm.its(
    cluster.dtrajs, lags=[1, 2, 5, 10, 20, 50], nits=4, errors='bayes')

fig, axes = plt.subplots(1, 3, figsize=(12, 3))
pyemma.plots.plot_feature_histograms(tica_concatenated, ax=axes[0])
pyemma.plots.plot_density(*tica_concatenated[:, :2].T, ax=axes[1], cbar=False, alpha=0.1)
axes[1].scatter(*cluster.clustercenters[:, :2].T, s=15, c='C1')
axes[1].set_xlabel('IC 1')
axes[1].set_ylabel('IC 2')
pyemma.plots.plot_implied_timescales(its, ax=axes[2], units='ps')
fig.tight_layout()

#### Exercise 2
Let's see if your discretized data is suitable to converge five slow implied timescales using a bayesian HMM.

In [ ]:
pyemma.plots.plot_implied_timescales #FIXME

###### Solution

In [ ]:
pyemma.plots.plot_implied_timescales(
    pyemma.msm.timescales_hmsm(
        cluster.dtrajs, 6, lags=[1, 2, 3, 4], errors='bayes'), units='ps');

#### Exercise 3
Estimate a bayesian HMM and perform a CK test.

In [ ]:
hmm = #FIXME
pyemma.plots. #FIXME

###### Solution

In [ ]:
hmm = pyemma.msm.bayesian_hidden_markov_model(cluster.dtrajs, 6, lag=1, dt_traj='1 ps')
pyemma.plots.plot_cktest(hmm.cktest(mlags=2), units='ps');

#### Exercise 4
Now that you have a model, be creative and visualize the metastable regions in your projected space.

In [ ]:
#FIXME

###### Solution

In [ ]:
def draw_panel(ax, i, j):
    _, _, misc = pyemma.plots.plot_state_map(
        *tica_concatenated[:, [i, j]].T,
        hmm.metastable_assignments[dtrajs_concatenated],
        ax=ax)
    ax.set_xlabel('IC {}'.format(i + 1))
    ax.set_ylabel('IC {}'.format(j + 1))
    misc['cbar'].set_ticklabels(range(1, hmm.nstates + 1))

fig, axes = plt.subplots(2, 2, figsize=(10, 8))
draw_panel(axes[0, 0], 0, 2)
draw_panel(axes[0, 1], 1, 2)
draw_panel(axes[1, 0], 0, 1)
axes[1, 1].set_axis_off()
fig.tight_layout()

# Supplementary Cells

Generate and plot implied time-scales plots for Bayesian Hidden Markov models

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharey=True)
pyemma.plots.plot_implied_timescales(
    pyemma.msm.timescales_hmsm(
        cluster.dtrajs, 4, lags=[1, 2, 3, 6], errors='bayes', nsamples=50),
    ax=axes[0], units='ps')
pyemma.plots.plot_implied_timescales(
    pyemma.msm.timescales_hmsm(
        cluster.dtrajs, 6, lags=[1, 2, 3, 6], errors='bayes', nsamples=50),
    ax=axes[1], units='ps')
fig.tight_layout()
#fig.savefig('_misc/hmm_its_4and6_states.png')

Generate 6-state Bayesian HMM, compute CK-test and plot results

In [ ]:
hmm_6 = pyemma.msm.bayesian_hidden_markov_model(cluster.dtrajs, 6, lag=1, dt_traj='1 ps', nsamples=50)
fig, _ = pyemma.plots.plot_cktest(hmm_6.cktest(mlags=2), units='ps');
fig.savefig('_misc/hmm_ck_6states.png')

## Wrapping up

In this notebook, we have learned how to use a hidden Markov state model (HMM) and how they differ from an MSM.
In detail, we have used
- `pyemma.msm.timescales_hmsm()` function to obtain an implied timescale object for HMMs,
- `pyemma.msm.estimate_hidden_markov_model()` to estimate a regular HMM,
- `pyemma.msm.bayesian_hidden_markov_model()` to estimate a Bayesian HMM, 
- the `metastable_assignments` attribute of an HMM object to access the metastable membership of discrete states, 
- the `hidden_state_probabilities` attribute to assess probabilities of hidden states over time, and
- the `hidden_state_trajectories` attribute that extracts the most likely trajectory in hidden state space.

## References

<a id="cite-hmm-baum-welch-alg"/><sup><a href=#ref-1>[^]</a></sup>Leonard E. Baum and Ted Petrie and George Soules and Norman Weiss. 1970. _A Maximization Technique Occurring in the Statistical Analysis of Probabilistic Functions of Markov Chains_. [URL](http://www.jstor.org/stable/2239727)

<a id="cite-noe-proj-hid-msm"/><sup><a href=#ref-2>[^]</a></sup>Frank Noé and Hao Wu and Jan-Hendrik Prinz and Nuria Plattner. 2013. _Projected and hidden Markov models for calculating kinetics and metastable states of complex molecules_. [URL](https://doi.org/10.1063/1.4828816)

<a id="cite-hmm-tutorial"/><sup><a href=#ref-3>[^]</a></sup>L.R. Rabiner. 1989. _A tutorial on hidden Markov models and selected applications in speech recognition_. [URL](https://doi.org/10.1109/5.18626)

